In [24]:
from yaml import safe_load

In [25]:
with open('sources.yml', 'r') as file:
    data = safe_load(file)
    print(type(data))

<class 'dict'>


In [ ]:
def create_data_list(source_file):
    with open(source_file, 'r') as file:
        data = safe_load(file)
    return data

In [3]:
from src.create_data_list import create_data_list

In [19]:
urls = create_data_list('sources.yml')

In [26]:
import pandas as pd
dtype = {
    "ANMOIS" : "str",
    "APT" : "str",
    "APT_NOM" : "str",
    "APT_ZON" : "str"
}
df = pd.read_csv(list(urls['airports'].values())[0], sep=';', dtype=dtype)
df['an'] = df['ANMOIS'].str.replace('..$', "", regex = True)
df['mois'] = df['ANMOIS'].str.replace('....*', "", regex = True)
df


,ANMOIS,APT,APT_NOM,APT_ZON,APT_PEQ,APT_PAX_dep,APT_PAX_arr,APT_PAX_tr,APT_FRP_dep,APT_FRP_arr,APT_NMVT_mxt,APT_NMVT_cgo,an,mois
0,201801,FMCZ,MAYOTTE-MARCEL HENRY,OM,35209,13108,18977,1202,"21,8","170,4",546,1,2018,1
1,201801,FMEE,LA REUNION-ROLAND GARROS,OM,269383,112430,127167,4768,743,"1758,7",1149,16,2018,1
2,201801,FMEP,ST-PIERRE-PIERREFONDS,OM,13219,6034,7181,0,0,"0,4",208,0,2018,1
3,201801,LFAQ,ALBERT-BRAY,MT,2108,119,124,0,108,"78,5",32,15,2018,1
4,201801,LFBA,AGEN-LA GARENNE,MT,2513,1335,1178,0,0,0,131,0,2018,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1057,201812,TFFG,ST-MARTIN-GRAND-CASE,OM,16778,8596,7789,7,"19,2","19,4",577,63,2018,12
1058,201812,TFFJ,ST-BARTHELEMY,OM,17435,8179,8875,0,"7,9","30,2",3069,113,2018,12
1059,201812,TFFR,POINTE-A-PITRE-LE RAIZET,OM,242057,107149,115766,6529,"275,2","986,1",2509,50,2018,12
1060,201812,ZZMT,AUTRES APT (MT),MT,5184,2302,2677,13,"14,4","4,8",1606,16,2018,12


In [22]:
def clean_dataframe(df):
    df['an'] = df['ANMOIS'].str.replace('..$', "", regex = True)
    df['mois'] = df['ANMOIS'].str.replace('....0*', "", regex = True)
    df = df.rename(str.lower, axis="columns")
    return df

df_1 = clean_dataframe(df)
trafic = df_1['apt_pax_dep'] + df_1['apt_pax_tr'] + df_1['apt_pax_arr']
trafic



0        33287
1       244365
2        13215
3          243
4         2513
         ...  
1057     16392
1058     17054
1059    229444
1060      4992
1061     14078
Length: 1062, dtype: int64

In [ ]:
def import_compagnies_data(list_files):
    col_types = {
        "ANMOIS" = "str",
        "CIE" = "str",
        "CIE_NOM" = "str",
        "CIE_NAT" = "str",
        "CIE_PAYS" = "str"
    }
    temp = []
    for file in list_files:
        df = pd.read_csv(file, sep=';', dtype=dtype)
        temp.append(clean_dataframe(df))
    return pd.concat(temp)

In [ ]:
def import_liaisons_data(list_files):
    col_types = {
        "ANMOIS": "str",
        "LSN": "str",
        "LSN_DEP_NOM": "str",
        "LSN_ARR_NOM": "str",
        "LSN_SCT": "str",
        "LSN_FSC": "str"
    }
    temp = []
    for file in list_files:
        df = pd.read_csv(file, sep=';', dtype=dtype)
        temp.append(clean_dataframe(df))
    return pd.concat(temp)

In [15]:
import geopandas as gpd

airports_location = gpd.read_file(urls['geojson']['airport'], sep=';')
airports_location

,Nom,Code.IATA,Code.OACI,anmois,apt_nom,apt_zon,apt_peq,apt_pax_dep,apt_pax_arr,apt_pax_tr,apt_frp_dep,apt_frp_arr,apt_nmvt_mxt,apt_nmvt_cgo,an,mois,trafic,volume,color,geometry
0,TUBUAI MATAURA,TUB,NTAT,202105.0,TUBUAI MATAURA,OM,1721.0,614.0,674.0,379.0,0.6,4.8,36.0,0.0,2021.0,05,1667.0,1,darkred,POINT (-149.52000 -23.37000)
1,ILE DES PINS,ILP,NWWE,202105.0,ILE DES PINS,OM,7122.0,3491.0,3495.0,0.0,2.8,10.9,146.0,0.0,2021.0,05,6986.0,2,forestgreen,POINT (167.46000 -22.59000)
2,RURUTU,RUR,NTAR,202105.0,RURUTU,OM,1605.0,547.0,548.0,470.0,0.4,3.6,36.0,0.0,2021.0,05,1565.0,1,darkred,POINT (-151.36000 -22.43000)
3,NOUMEA MAGENTA,GEA,NWWM,202105.0,NOUMEA MAGENTA,OM,34476.0,16597.0,17014.0,0.0,64.0,22.5,861.0,2.0,2021.0,05,33611.0,3,dodgerblue,POINT (166.47000 -22.26000)
4,NOUMEA LA TONTOUTA,NOU,NWWW,202105.0,NOUMEA LA TONTOUTA,OM,5946.0,1497.0,1026.0,0.0,82.4,259.9,71.0,11.0,2021.0,05,2523.0,2,forestgreen,POINT (166.21000 -22.01000)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,PARIS-LE BOURGET,LBG,LFPB,202105.0,PARIS-LE BOURGET,MT,6959.0,3403.0,3556.0,0.0,0.0,0.0,3401.0,0.0,2021.0,05,6959.0,2,forestgreen,POINT (2.44000 48.97000)
60,PARIS-CHARLES DE GAULLE,CDG,LFPG,202105.0,PARIS-CHARLES DE GAULLE,MT,2754060.0,543015.0,578140.0,89.0,79952.5,83329.2,11503.0,3363.0,2021.0,05,1121244.0,4,gold,POINT (2.55000 49.01000)
61,CAEN-CARPIQUET,CFR,LFRK,202105.0,CAEN-CARPIQUET,MT,20166.0,10738.0,9426.0,2.0,0.0,0.0,269.0,0.0,2021.0,05,20166.0,3,dodgerblue,POINT (-0.45000 49.17000)
62,BEAUVAIS-TILLE,BVA,LFOB,202105.0,BEAUVAIS-TILLE,MT,74030.0,37938.0,35645.0,447.0,0.0,0.0,650.0,0.0,2021.0,05,74030.0,4,gold,POINT (2.11000 49.45000)


In [18]:
import folium

m = folium.Map()
folium.GeoJson(airports_location).add_to(m)
m